In [ ]:
from tensorflow import keras


In [ ]:
from CustomLayers import *
from GS_composer import *
from Functions import *
from ClassModel import *

In [ ]:
Model_path = "C:/Users/uqcche32/OneDrive - The University of Queensland/PhD/HPC_Results/Sugarcane_disease/ML/1Head_RealForm32_32LocalB"
#Model_path = "E:/learning resource/OneDrive - The University of Queensland/PhD/HPC_Results/Sugarcane_disease/ML/"

In [ ]:
from keras import backend as K

In [ ]:
model_folder = "pachy_MultiHeadAttentionLNN_1"
#model_folder = "test_model"
full_path = Model_path + "/" + model_folder
model = keras.models.load_model(full_path,custom_objects={"MultiHead_QKV_BlockAttention": MultiHead_QKV_BlockAttention,
                                                         "SNPBlockLayer":SNPBlockLayer})

In [ ]:
model.summary()
#model = keras.Model(inputs=input1, outputs=QV_output)
model.compile(optimizer="RMSprop", loss="mean_squared_error")

In [ ]:

data = np.ones((1, 26086, 1))
model.predict(data)

In [ ]:
intermediate_layer_model = tf.keras.Model(inputs=model.input,
                                         outputs=model.get_layer('add_3').output)

In [ ]:
intermediate_output = intermediate_layer_model.predict(data)

In [ ]:
attention = intermediate_output[0]
attention = K.expand_dims(attention,axis=1)
print(attention.shape)

In [ ]:
LNN_weight = model.layers[2].get_weights()

In [ ]:
LNN_weight[0].shape

In [ ]:
SNP_attention = tf.multiply(attention,LNN_weight[0])

In [ ]:
SNP_attention = tf.reshape(SNP_attention,(26080,1,32))

In [ ]:
mean_SNP_attention = SNP_attention #tf.reduce_mean(SNP_attention,axis=2)
mean_SNP_attention.shape
mean_SNP_attention = np.array(tf.reshape(mean_SNP_attention,(26080*32,)))

In [ ]:
import matplotlib.pyplot as plt
import math

In [ ]:
print(mean_SNP_attention.shape)
plt.scatter(x=np.tile(np.arange(0,26080),32),y=abs((mean_SNP_attention)))

In [ ]:
mean_SNP_attention.shape

In [ ]:
def export_attention_score_by_trait(filename,trait,val):
    model = keras.models.load_model(filename,custom_objects={"MultiHead_QKV_BlockAttention": MultiHead_QKV_BlockAttention,
                                                         "SNPBlockLayer":SNPBlockLayer})
    model.summary()
    #model = keras.Model(inputs=input1, outputs=QV_output)
    model.compile(optimizer="RMSprop", loss="mean_squared_error")
    data = np.ones((1, 26086, 1))
    intermediate_layer_model = tf.keras.Model(inputs=model.input,
                                         outputs=model.get_layer('add_3').output)
    attention = intermediate_output[0]
    attention = K.expand_dims(attention,axis=1)
    print(attention.shape)
    
    LNN_weight = model.layers[2].get_weights()
    SNP_attention = tf.multiply(attention,LNN_weight[0])
    SNP_attention = np.array(tf.reshape(SNP_attention,(32,26080)))
    SNP_attention = pd.DataFrame(SNP_attention,columns=list(range(1,SNP_attention.shape[1]+1)))
    SNP_attention.shape
    SNP_attention.insert(0,'Channel',range(1,33))
    SNP_attention.insert(0,'Val',val)
    SNP_attention.insert(0,"Trait",trait)
    
    SNP_attention = SNP_attention.values.tolist()
    #mean_SNP_attention = [trait,val]+list(np.array(tf.reshape(mean_SNP_attention,(26080,))))
    #mean_SNP_attention = pd.DataFrame([trait,val]+list(mean_SNP_attention),columns=[trait,val]+list(range(1,length(mean_SNP_attention))))

    return SNP_attention
    
    

In [ ]:
Model_path = "C:/Users/uqcche32/OneDrive - The University of Queensland/PhD/HPC_Results/Sugarcane_disease/ML/1Head_RealForm32_32LocalB/"
records = []
for trait in ["smut","pachy"]:
    for val in range(1,6):
        filename = Model_path + trait + "_MultiHeadAttentionLNN_"+str(val)
        record = export_attention_score_by_trait(filename,trait,val)
        records.append(record)


In [ ]:
recordss = []
for i in records:
    for j in i:
        recordss.append(j)
records_df = pd.DataFrame(recordss,columns=["Trait","Val","Channel"]+list(range(1,26081)))
output_path = Model_path + "Attention_score_bychannel.csv"
records_df.to_csv(output_path,sep="\t",index=False)

In [ ]:
recordss[0]

In [ ]:
records.insert(0,"Val", #columns.values[0:2] = ["Trait","Val"]
output_path = Model_path + "Attention_score.csv"
records.to_csv(output_path,sep="\t",index=False)

In [ ]:
records_df.head(4)